<a href="https://colab.research.google.com/github/jonofficial/Resources/blob/main/Docker-notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# WHAT IS DOCKER?
# A platform for building, running and shipping applications
# you may encounter cases when an application that one has developed runs on one machine but not on others
# this might be due to reasons like:
# one or more files missing
# software version mismatch (target machine may run diff version)
# different configuration settings

# docker enables:
# 1. consistency across environments (ensures an application runs the same on any OS and any machine)
# 2. Isolation (isolates our app from the dependencies, so no more clashes between applications)
# 3. Portability (easily moves app from testing to development to other processess)
# 4. version control (helps us track our app, can return to previous version when anything goes wrong)
# 5. scalability (easily creates copy of our application when needed)
# 6. DevOps integration

# HOW DOES DOCKER WORK?
# mainly depends on 2 concepts: images and containers
# An image is a lightweight, standalone, executable package that includes everything to run a piece of software including the code, runtime, OS,libraries, system-tools
# A container is a runnable instance of a docker image
# we can run multiple containers from a single image
# analogically docker image is the recipe and container is the cake that u cook from recipe.
# container - an isolated environment for running an application
# virtual machine - an abstraction of a machine (physical hardware)

# DOCKER VOLUME
# A docker volume is a persistent data storage mechanism that allows data to be shared between a dock container and a host machine
# volume ensures data durability and persistence even if the container is stopped or removed
# think of it as a shared folder or storage container compartment of container

# DOCKER NETWORK
# a communication channel to enable containers to talk to each other or with external world (enables container to share info and services while maintaining isolation)

# DOCKER WORKFLOW (divided into 3 parts)
# 1. Docker client (CLI or GUI) - where user gives command
# 2. Docker Host (Docker Daemon) - runs the container
# 3. Docker registry (Docker Hub) - docker images are stored

# you can checkout docker hub in google for public images
# if we wanna create an image of our own, then we do it by docker file

# to create an public image
docker pull ubuntu

# to create and run a container from image
docker run -it ubuntu
# close terminal to quite

# to create our own image
# first create a folder (docker-course), inside the course create another folder hello-docker
# create two files inside hello-docker:
Dockerfile
hello.js

# inside Dockerfile type the below commands
FROM node:20-alpine

WORKDIR /app

COPY . .

CMD node hello.js

# open terminal
cd hello-docker
docker build -t hello-docker .

# to check whether the image is created
docker images

# to build container from the created image
docker run hello-docker

# to run the container
docker run -it hello-docker sh
node hello.js

# download docker-desktop for GUI, u can see containers and images in the desktop app

# to do some complex testing, create a react vite project (the objective here is that instead of installing all the node modules with "npm i", we are dockerising the project to ship all the required modules)
npm create vite@latest react-docker
# select react+TS
cd react-docker
# create a Dockerfile inside react-docker folder

# INSIDE DOCKERFILE

# set the base image to create the image for react app
FROM node:20-alpine

# create a user with permissions to run the app
# -S -> create a system user
# -G -> add the user to a group
# This is done to avoid running the app as root
# If the app is run as root, any vulnerability in the app can be exploited to gain access to the host system
# It's a good practice to run the app as a non-root user
RUN addgroup app && adduser -S -G app app

# set the user to run the app
USER app

# set the working directory to /app
WORKDIR /app

# copy package.json and package-lock.json to the working directory
# This is done before copying the rest of the files to take advantage of Docker’s cache
# If the package.json and package-lock.json files haven’t changed, Docker will use the cached dependencies
COPY package*.json ./

# sometimes the ownership of the files in the working directory is changed to root
# and thus the app can't access the files and throws an error -> EACCES: permission denied
# to avoid this, change the ownership of the files to the root user
USER root

# change the ownership of the /app directory to the app user
# chown -R <user>:<group> <directory>
# chown command changes the user and/or group ownership of for given file.
RUN chown -R app:app .

# change the user back to the app user
USER app

# install dependencies
RUN npm install

# copy the rest of the files to the working directory
COPY . .

# expose port 5173 to tell Docker that the container listens on the specified network ports at runtime
EXPOSE 5173

# command to run the app
CMD npm run dev

# END OF DOCKER FILE

# create a new file inside react-docker
.dockerignore

# to build an image
docker build -t react-docker .

# to use the image, we have to run it
docker run react-docker

# but when we go to localhost: 5173, the site isnt running because we have to map the port of container to port of docker (even if we commanded docker to listen to port 5173 using EXPOSE in Dockerfile)
# to expose host navigate to package.json
dev: "vite --host"; # make the changes add "--host"

# go to terminal and type the command
docker run -p 5173:5173 react-docker # if it doesnt run, then clear the running containers and other useless containers using below

# to see container ID's and their images
docker ps # only shows active/running containers

# to all containers
docker ps -a

# to stop a running container
docker stop docker-name # either u can use the container name or its ID (first 3 letters of its ID is enough)

# to remove all inactive containers
docker container prune

# to remove specific container
docker rm docker-name/ID

# to remove a runnning container use the above container with --force
docker run docker-name --force

# after containerising the project, when we make changes in the code base, the container doesnt get updated. so to keep the container updated
docker run -p 5173:5173 -v "$(pwd):/app" -v /app/node_modules react-docker

# PUBLISHING A DOCKER IMAGE
cd react-docker
docker login # if already logged in docker-desktop then it should automatically authenticate you

# to publish
docker tag react-docker username/react-docker # username can be found from docker-desktop
docker push username/react-docker

# DOCKER COMPOSE
# running all these commands to build an image, a container out of the image and running the container and mapping them to host - is tedious
# we can automate all these using docker compose using one straight-forward command
# it uses a yaml file to configure the services,networks,volumes for your application
# we dont have to run 10 commands for 10 containers for your application
# we can put all the info in a single file and a single command to run the file
# the file can be put manually or docker provides a CLI called docker-init to generate these files for us
# with docker-init we initialize our app with all the needed files needed to dockerize it by specifying tech choices

# create a new vite project
cd vite-project
docker init

# it asks some few questions about tech choices, select appropriately and it automatically creates new files for us
Dockerfile
.dockerignore
compose.yaml

# go to compose.yaml and change server to web, remove environmental variables if not used
# add volumes to the yaml file

services:
  web:
    build:
      context: .
    ports:
      - 5173:5173
    volumes:
      - .:/app
      - /app/node_modules

# now run
docker compose up

# DOCKER COMPOSE WATCH  (new feature of docker) - most optimal for developers today
# but eventhough docker compose helps us make the code base upto date, it still makes us create new containers to run when package files are changed (it automatically stops old containers when new ones are created)
# to solve the problem we have docker compose watch
# 3 things can be done using this - sync, rebuild, sync-restart

# DOCKER SCOUT (new feature)
# when we are creating images for our application, we are essentially stacking layers of existing images
# but some of the images may have security vulnerabilities making our application prone to attacks
# docker scout is a tool that helps us be proactive about our security - scans all the images and creates a detailed list called SBOM (software bill of materials)
# then docker scout checks the list against the always updated databases of known vulnerabilities. if it finds any , it lets us know

view packages and CVE's button - complete report analysis of our image